In [7]:
import open3d as o3d
import numpy as np

from tools.camera_params import CameraParams
from tools.data_reader import Observation, get_observation

import matplotlib.pyplot as plt

import copy

In [8]:
def get_translation_matrix(obs_start: Observation, obs_goal: Observation) -> np.ndarray:
    camera_params = CameraParams()
    pinhole_camera_intrinsic = o3d.camera.PinholeCameraIntrinsic()
    pinhole_camera_intrinsic.set_intrinsics(640, 480, camera_params.fx, camera_params.fy, camera_params.cx, camera_params.cy)
    start_color = o3d.geometry.Image(obs_start.image)
    start_depth = o3d.geometry.Image(obs_start.depth)
    goal_color = o3d.geometry.Image(obs_goal.image)
    goal_depth = o3d.geometry.Image(obs_goal.depth)
    start_rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(start_color, start_depth, depth_scale=0.05)
    goal_rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(goal_color, goal_depth, depth_scale=0.05)

    option = o3d.pipelines.odometry.OdometryOption()
    odom_init = np.identity(4)

    _, trans_hybrid_term, _ = o3d.pipelines.odometry.compute_rgbd_odometry(
        start_rgbd, goal_rgbd, pinhole_camera_intrinsic,
        odom_init, o3d.pipelines.odometry.RGBDOdometryJacobianFromHybridTerm(), option)

    return trans_hybrid_term

In [10]:

for i in range(10):
    mat = get_translation_matrix(get_observation(170 + i, dataset_folder="datasets/rgbd_dataset_freiburg1_floor"), get_observation(171 + i, dataset_folder="datasets/rgbd_dataset_freiburg1_floor"))
    print(mat)

[[ 0.99967385  0.0222413  -0.01255029 -0.02608513]
 [-0.02226994  0.99974969 -0.00214664  0.00312602]
 [ 0.0124994   0.00242543  0.99991894  0.00139866]
 [ 0.          0.          0.          1.        ]]
[[ 0.99934499  0.03422252 -0.01176517 -0.01985292]
 [-0.03424814  0.99941139 -0.0019834   0.00508129]
 [ 0.01169037  0.00238504  0.99992882  0.00141102]
 [ 0.          0.          0.          1.        ]]
[[ 9.99235934e-01  3.76925276e-02 -1.03354864e-02 -1.54782582e-02]
 [-3.76946519e-02  9.99289304e-01 -1.07428444e-05 -4.80272069e-04]
 [ 1.03277361e-02  4.00327199e-04  9.99946587e-01  1.84875477e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[ 9.99417667e-01  3.28148118e-02 -9.35491356e-03 -1.75071305e-02]
 [-3.28190618e-02  9.99461264e-01 -3.01122879e-04 -8.20743248e-04]
 [ 9.33999245e-03  6.07967012e-04  9.99956196e-01  1.54365340e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[ 9.99655808e-01  2.47614579e-02 -8.66810136e-03 -1